In [1]:
from pondmemory.database.Mongo import Mongo
mongo = Mongo('localhost', 27017, 'PondMemoryDB')

In [2]:
m_collection = "WechatMsg"
res = list(mongo.find(m_collection, {"Type":1}))

2024-01-31 21:33:06,402 - INFO: 与MongoDB localhost:27017 建立连接


In [3]:
import jieba 
jieba.load_userdict("pondmemory/cut_words/SogouLabDic.txt")
jieba.load_userdict("pondmemory/cut_words/dict_baidu_utf8.txt")
jieba.load_userdict("pondmemory/cut_words/dict_pangu.txt")
jieba.load_userdict("pondmemory/cut_words/dict_sougou_utf8.txt")
jieba.load_userdict("pondmemory/cut_words/dict_tencent_utf8.txt")



Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\killuayz\AppData\Local\Temp\jieba.cache
Loading model cost 0.658 seconds.
Prefix dict has been built successfully.


In [4]:
jieba.load_userdict("pondmemory/cut_words/my_dict.txt")

In [5]:
stopwords = {}.fromkeys([ line.rstrip() for line in open('pondmemory/cut_words/Stopword.txt', encoding='utf8') ])

In [6]:
cut_words_list = []
for msg in res:
    content = msg["StrContent"]
    seg_list = list(jieba.cut(content, cut_all=False))
    for seg in seg_list:
        if seg not in stopwords and not seg.isspace():
            cut_words_list.append(seg)

In [7]:
cut_words_map_cnt = {}
for word in cut_words_list:
    if word not in cut_words_map_cnt:
        cut_words_map_cnt[word] = 1
    else:
        cut_words_map_cnt[word] += 1

cut_words_map_cnt_list = list(cut_words_map_cnt.items())
cut_words_map_cnt_list.sort(key=lambda x: x[1], reverse=True)
# print(cut_words_map_cnt_list)
for i in range(min(20, len(cut_words_map_cnt_list))):
    print(f"#{i+1:<10d} {cut_words_map_cnt_list[i][0]:<15s} {cut_words_map_cnt_list[i][1]:<10d}")
    

#1          xinxin          3918      
#2          可怜              3458      
#3          害羞              2396      
#4          阳哥              1709      
#5          旺柴              1308      
#6          破涕为笑            1261      
#7          亲亲              1120      
#8          晚安              1117      
#9          捂脸              1088      
#10         呆头鹅             833       
#11         dte             776       
#12         dde             711       
#13         苦涩              687       
#14         流泪              507       
#15         yg              490       
#16         猫宁              488       
#17         明天              483       
#18         呆呆鹅             410       
#19         衰               352       
#20         今晚              336       


In [8]:
from pondmemory.sentiment.evaulate_sentiment import evaulate_sentiment
pos_words_list, neg_words_list = evaulate_sentiment(cut_words_list)

100%|██████████| 161126/161126 [00:19<00:00, 8207.04it/s]


In [9]:
pos_words_list

[('😁', 0.5),
 ('我想', 0.5441789912779675),
 ('米', 0.5038759689922481),
 ('测', 0.7499999999999999),
 ('补测', 0.5),
 ('机会', 0.7541666666666667),
 ('苦涩', 0.875),
 ('苦涩', 0.875),
 ('苦涩', 0.875),
 ('苦涩', 0.875),
 ('苦涩', 0.875),
 ('苦涩', 0.875),
 ('我刚刚', 0.5918367346938779),
 ('老师', 0.8788819875776398),
 ('看看你', 0.5262327818078083),
 ('缓测', 0.5),
 ('后续', 0.5999999999999999),
 ('统一', 0.6785714285714288),
 ('组织', 0.7272727272727275),
 ('测试', 0.7244094488188978),
 ('等一下', 0.5262327818078083),
 ('要样', 0.5),
 ('缓测', 0.5),
 ('嘿哈', 0.5262327818078083),
 ('你可以', 0.5262327818078083),
 ('缓测', 0.5),
 ('过会儿', 0.7499999999999999),
 ('哦哦哦', 0.5262327818078083),
 ('好滴', 0.6644044980400812),
 ('我猜', 0.6578947368421054),
 ('概率', 0.5),
 ('会让你', 0.5539033457249072),
 ('通过的', 0.5262327818078083),
 ('感谢', 0.6911764705882352),
 ('合十', 0.5),
 ('合十', 0.5),
 ('体委', 0.5),
 ('就去', 0.5131611932815242),
 ('试', 0.59375),
 ('对了', 0.5262327818078083),
 ('记得', 0.7848837209302325),
 ('更要', 0.6241768579492005),
 ('下次', 0.5689851

In [11]:
pos_words_list.sort(lambda x:x[1], reverse=True)
neg_words_list.sort(lambda x:x[1], reverse=False)

with open("./neg_words_list.txt", "w", encoding="utf8") as t:
    for word, rate in neg_words_list:
        t.write(f"{word}\t{rate}\n")

with open("./pos_words_list.txt", "w", encoding="utf8") as t:
    for word, rate in pos_words_list:
        t.write(f"{word}\t{rate}\n")

TypeError: sort() takes no positional arguments